In [34]:
import os

import re
import pymorphy2
from typing import List

import pandas as pd

In [2]:
# configs
PROJECT_PATH = "/home/alex/paper-2025-anonymous-submission"

In [3]:
dataset = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/processed_data/dataset.json"
    ),
    orient="index"
)

In [4]:
dataset = dataset.dropna(subset=["annotations"])

In [5]:
def parse_one_annotation(annot):

    searcheable_elements = list()
    searcheable_links = list()

    if isinstance(annot["reference_string"], list):
        for i in range(len(annot["reference_string"])):
            searcheable_elements.append(
                annot["reference_string"][i]
            )
    elif isinstance(annot["reference_string"], str) and annot["reference_string"].strip() != "":
        searcheable_elements.append(
                annot["reference_string"]
        )

    if isinstance(annot["reference_url"], list):
        for i in range(len(annot["reference_url"])):
            searcheable_links.append(
                annot["reference_url"][i]
            )
    elif isinstance(annot["reference_url"], str) and annot["reference_url"].strip() != "":
        searcheable_links.append(
                annot["reference_url"]
        )

    return searcheable_elements, searcheable_links

In [6]:
def extract_searcheable_elements_and_links(annotations):

    searcheable_elements = list()
    searcheable_links = list()

    for annot in annotations:

        if isinstance(annot, dict):

            tmp_elements, tmp_links = parse_one_annotation(annot)
            searcheable_elements.extend(
                tmp_elements
            )
            searcheable_links.extend(
                tmp_links
            )

        elif isinstance(annot, list):

            for single_annot in annot:
                tmp_elements, tmp_links = parse_one_annotation(single_annot)
                searcheable_elements.extend(
                    tmp_elements
                )
                searcheable_links.extend(
                    tmp_links
                )
    
    return searcheable_elements, searcheable_links
                    


In [7]:
dataset["annotations_len"] = dataset["annotations"].apply(lambda x: len(x))

In [8]:
dataset["searcheable_elements"] = dataset["annotations"].apply(lambda x: extract_searcheable_elements_and_links(x)[0])
dataset["searcheable_links"] = dataset["annotations"].apply(lambda x: extract_searcheable_elements_and_links(x)[1])

In [9]:
dataset["searcheable_elements_len"] = dataset["searcheable_elements"].apply(lambda x: len(x))
dataset["searcheable_links_len"] = dataset["searcheable_links"].apply(lambda x: len(x))

In [13]:
dataset = dataset.query("searcheable_elements_len >= 1 or searcheable_links_len >= 1")

In [32]:
def normalize_link(link):
    if "https://ru.wikipedia.org" in link:
        link = link.split("https://ru.wikipedia.org/wiki/")[1]
    elif "https://ru.wiktionary.org" in link:
        link = link.split("https://ru.wiktionary.org/wiki/")[1]
    
    if "#" in link:
        position = link.find("#")
        link = link[:position]
    
    link = re.sub(r'\(.*?\)', '', link).strip()
    print(link)

    link = re.sub(r'[^a-zA-Zа-яА-ЯёЁ]', ' ', link)
    link = re.sub(r'\s+', ' ', link).strip()

    return link.lower()

In [38]:
def tokenize(text: str) -> List[str]:
    """Разбивает текст на токены (слова)."""
    return re.findall(r'\b\w+\b', text.lower())

def lemmatize(tokens: List[str]) -> List[str]:
    """Лемматизирует список токенов."""
    morph = pymorphy2.MorphAnalyzer()
    return [morph.parse(token)[0].normal_form for token in tokens]

def normalize_text(text):
    tokens = tokenize(text)
    lemmas = lemmatize(tokens)
    return " ".join(lemmas)

In [39]:
normalize_text("Всё познаётся в сравнении")

'всё познаваться в сравнение'

In [14]:
dataset.sample(10)

,annotations,summary,is_word_play,date,article_url,headline,lead,annotations_len,searcheable_elements,searcheable_links,searcheable_elements_len,searcheable_links_len
1754,[{'headline_substring': 'Все подается в сравне...,“Ъ” ознакомился с рекомендациями по работе с и...,True,2022-11-07,https://www.kommersant.ru/doc/5457819,Все подается в сравнении,Единороссам посоветовали найти аналогии между ...,1,[Всё познаётся в сравнении],[],1,0
1030,"[{'headline_substring': 'национальный вопрос',...",Заявление Всеанглийского лаун-теннисного клуба...,True,2022-04-23,https://www.kommersant.ru/doc/5326302,Теннисистов испортил национальный вопрос,АТР и WTA готовят меры по защите интересов игр...,1,[квартирный вопрос],[https://ru.wikipedia.org/wiki/Квартирный_вопр...,1,1
639,"[{'headline_substring': 'В погоне за ставкой',...",В неблагоприятных рыночных условиях Минфин уст...,True,2023-08-09,https://www.kommersant.ru/doc/6149324,В погоне за ставкой,Минфин не хочет платить высокую премию инвесторам,1,"[""В погоне за счастьем""]",[https://ru.wikipedia.org/wiki/В_погоне_за_сча...,1,1
771,"[{'headline_substring': 'складах', 'start_inde...",Высокий спрос со стороны ритейлеров и дистрибу...,True,2021-02-08,https://www.kommersant.ru/doc/4927037,С логопарками не в складах,Девелоперы не успевают строить инфраструктурны...,1,[ладах],[],1,0
1015,"[{'headline_substring': 'За синие горы, за бел...",ОАО РЖД наметило три варианта железной дороги ...,True,2021-09-08,https://www.kommersant.ru/doc/4936100,"За синие горы, за белый туман",ОАО РЖД наметило три маршрута в Китай через Го...,1,"[За Синие Горы, за белый туман]",[],1,0
1165,"[{'headline_substring': 'Сами себе стратеги', ...",Вчера комиссия Госдумы по вопросам поддержки м...,True,2021-01-09,https://www.kommersant.ru/doc/4966807,Сами себе стратеги,В Госдуме обсудили предложенные бизнесом инстр...,1,[Сам себе режиссер],[https://ru.wikipedia.org/wiki/Сам_себе_режиссёр],1,1
70,"[{'headline_substring': 'Осторожно, сети замед...",Средняя скорость мобильного интернета в москов...,True,2021-12-14,https://www.kommersant.ru/doc/5129336,"Осторожно, сети замедляются",Интернет в метро тормозит из-за роста трафика,1,"[Осторожно, двери закрываются]",[],1,0
1549,"[{'headline_substring': 'Прощание германки', '...",За месяц до парламентских выборов в Германии к...,True,2021-08-19,https://www.kommersant.ru/doc/4948649,Прощание германки,Ангела Меркель едет в Москву подвести итоги,1,[Прощание славянки],[https://ru.wikipedia.org/wiki/Прощание_славянки],1,1
1367,"[{'headline_substring': 'Саммит саммитом, а «в...","Россия и США продолжают «посольские войны», не...",True,2021-03-08,https://www.kommersant.ru/doc/4928280,"Саммит саммитом, а «война» по расписанию","Конфликт между Россией и США продолжается, нес...",1,"[Война войной, а обед по расписанию]",[],1,0
2209,"[{'headline_substring': 'Говорят, чемпион нена...",Семикратный чемпион мира «Формулы-1» Льюис Хэм...,True,2023-08-18,https://www.kommersant.ru/doc/6171338,"Говорят, чемпион ненастоящий",Фелипе Масса требует пересмотра итогов первенс...,1,"[Говорят, царь не настоящий]",[],1,0


In [ ]:


def tokenize(text: str) -> List[str]:
    """Разбивает текст на токены (слова)."""
    return re.findall(r'\b\w+\b', text.lower())

def lemmatize(tokens: List[str]) -> List[str]:
    """Лемматизирует список токенов."""
    morph = pymorphy2.MorphAnalyzer()
    return [morph.parse(token)[0].normal_form for token in tokens]

def preprocess_text(text: str) -> List[str]:
    """Токенизирует и лемматизирует текст."""
    tokens = tokenize(text)
    lemmas = lemmatize(tokens)
    return lemmas

if __name__ == "__main__":
    text = "Коты мурлыкают, а собаки лают!"
    result = preprocess_text(text)
    print(result)